In [1]:
from sigvisa.graph.sigvisa_graph import SigvisaGraph
from sigvisa.signals.common import Waveform





from sigvisa.source.event import get_event






from sigvisa.infer.run_mcmc import run_open_world_MH

from sigvisa.infer.mcmc_logger import MCMCLogger

In [2]:

from sigvisa.synthetic.doublets import *

def sample_events(basedir, seed=0):
    n_evs = 10
    lons = [129, 130]
    lats = [-3.5, -4.5]
    times = [1238889600, 1245456000]
    mbs = [4.0, 5.0]
    sw = SampledWorld(seed=seed)
    sw.sample_region_with_doublet(n_evs, lons, lats, times, mbs)
    sw.stas = ["MK31", "AS12", "CM16", "FITZ", "WR1"]
    gpcov = GPCov([0.7,], [ 40.0, 5.0],
                  dfn_str="lld",
                  wfn_str="compact2")
    param_means = build_param_means(sw.stas)
    sw.set_basis(wavelet_family="db4_2.0_3_30", iid_repeatable_var=0.1,
                  iid_nonrepeatable_var=0.4, srate=5.0)
    sw.joint_sample_arrival_params(gpcov, param_means)
    sw.sample_signals("freq_0.8_4.5")

    wave_dir = os.path.join(basedir, "sampled_%d_spreadtime" % seed)
    sw.serialize(wave_dir)
    #sw.train_gp_models_true_data()
    #sw.save_gps(wave_dir, run_name="synth_truedata")
    return sw


import os
basedir = os.path.join(os.getenv("SIGVISA_HOME"), "experiments", "synth_wavematch")
#sw = sample_events(basedir)

wave_dir = os.path.join(basedir, "sampled_%d_spreadtime" % 0)
sw = load_sampled_world(wave_dir)
dummyPrior = dict([(param, Gaussian(sw.param_means["MK31"][param], std=np.sqrt(sw.gpcov.wfn_params[0]))) for param in sw.param_means["MK31"].keys()
])

In [3]:
import cPickle as pickle
with open("../logs/mcmc/01687/step_000049/pickle.sg", 'rb') as f:
    sg = pickle.load(f)

In [4]:
"""for sta in sg._joint_gpmodels.keys():
    for param in sg._joint_gpmodels[sta].keys():
        sg._joint_gpmodels[sta][param]._clear_cache()
        
for sta in sg.station_waves.keys():
    for wn in sg.station_waves[sta]:
        wn.pass_jointgp_messages()
        
with open("../logs/mcmc/01687/step_000049/pickle.sg", 'wb') as f:
    pickle.dump(sg, f)"""

'for sta in sg._joint_gpmodels.keys():\n    for param in sg._joint_gpmodels[sta].keys():\n        sg._joint_gpmodels[sta][param]._clear_cache()\n        \nfor sta in sg.station_waves.keys():\n    for wn in sg.station_waves[sta]:\n        wn.pass_jointgp_messages()\n        \nwith open("../logs/mcmc/01687/step_000049/pickle.sg", \'wb\') as f:\n    pickle.dump(sg, f)'

In [5]:
print sg.current_log_p()


-61837.5760379


In [6]:
logger = MCMCLogger(write_template_vals=False, dump_interval=5)

corrupted_evs = []
for i,ev in enumerate(sw.evs):
    eid = i+1
    lon = sg.all_nodes["%d;lon_obs" %eid ].get_value()
    lat = sg.all_nodes["%d;lat_obs" %eid ].get_value()
    depth = sg.all_nodes["%d;depth_obs" %eid ].get_value()
    mb = sg.all_nodes["%d;mb_obs" %eid ].get_value()
    time = sg.all_nodes["%d;time_obs" %eid ].get_value()
    cev = Event(lon=lon, lat=lat, depth=depth, mb=mb, time=time, eid=eid)
    corrupted_evs.append(cev)
    
with open(os.path.join(logger.run_dir, "obs_events.pkl"), "wb") as f:
    pickle.dump(corrupted_evs, f)

with open(os.path.join(logger.run_dir, "events.pkl"), "wb") as f:
    pickle.dump(sw.evs, f)
    
#sg.seed = 1
run_open_world_MH(sg, steps=1000,
                  enable_template_moves=True,
                  enable_event_moves=True,
                  logger=logger,
                  enable_event_openworld=False,
                  enable_template_openworld=False)

step 1: lp -61843.79, accepted arrival_time: 0.190%, coda_decay: 0.140%, coda_height: 0.190%, evdepth: 0.550%, evloc: 0.700%, evloc_big: 0.200%, evmb: 0.950%, evtime: 0.600%, indep_peak: 0.000%, peak_decay: 0.230%, peak_offset: 0.220%, , uatemplates: 0 , events: 10 


KeyboardInterrupt: 

In [12]:
print sg.seed
" step 2 -61851.71"

1963817276
